In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

results_dir = "/mnt/storage/saikat/work/gwas-eQTL/gxpred_pilot/gtex_cardiogenics/z1"

dirs = os.listdir(results_dir)

from collections import defaultdict

failures = defaultdict(list)
by_prior = defaultdict(list)

all_r2_df = np.array([])
melted_r2_df = np.array([])
for prior in ["test_1KGannots_fixedPI"]: # "NoPriors_distfeat", 
    counter = 0
    subdirs = os.listdir(os.path.join(results_dir,prior))
    for sd in subdirs:
        _bslmm, _cutoff, _distfeat, _feat, pi, _mu, sigma, sigmabg, _tau = sd.split("_")
#         print(prior, pi, sigma, sigmabg)
        r2file = os.path.join(results_dir,prior,sd,"genes_r2.txt")
        if not os.path.exists(r2file):
            r2file = os.path.join(results_dir,prior,sd,"genes_r2_12.txt")
            if not os.path.exists(r2file):
                continue
        df = pd.read_table(r2file, header=0)
        newcolname = _cutoff+"_"+_distfeat+"_"+_feat+"_"+sigmabg+"_"+pi
        
        orig_df = df.copy()
        df=df.rename(columns = {'gxpred-bslmm':newcolname})
        # df.dropna(axis=0, subset=["Success"], inplace=True)
        counter += 1        
        
        orig_df["i_sigma"] = sigma
        orig_df["i_sigmabg"] = sigmabg
        orig_df["i_pi"] = pi
        orig_df["prior"] = prior
        orig_df["params"] = newcolname
        print(counter,prior,sd,"\t","Not Failures: ",np.sum(df.Success))
        
#         failures[d].append({sd:np.sum(df.Success)})
        # Merge all r2 values into one big df, one set of r2 per column
        if all_r2_df.shape[0] == 0:
            all_r2_df = df[["Ensembl_ID",newcolname]]
        else:
            all_r2_df = all_r2_df.merge(df[["Ensembl_ID",newcolname]], on="Ensembl_ID", how="outer")
        
        # Merge all r2 values in a melted fashion, including init params
        if melted_r2_df.shape[0] == 0:
            print("wii")
            melted_r2_df = orig_df.copy()
        else:
            melted_r2_df = pd.concat([melted_r2_df, orig_df],ignore_index=True )
